In [17]:
import numpy as np
from matplotlib import pyplot as plt
import sys, os, cv2, time

In [18]:
def extractChannels(Img):
    red = img[:,:,2]
    green = img[:,:,1]
    blue = img[:,:,0]
    return blue,green,red

In [19]:
def calcDerivativesForEachChannel(img):
    red = img[:,:,2]
    green = img[:,:,1]
    blue = img[:,:,0]

    #Calculate X derivatives
    I0x = cv2.Sobel(blue,cv2.CV_64F,1,0, ksize=3)
    I1x = cv2.Sobel(green,cv2.CV_64F,1,0, ksize=3)
    I2x = cv2.Sobel(red,cv2.CV_64F,1,0, ksize=3)

    #Caluclate Y derivatives
    I0y = cv2.Sobel(blue,cv2.CV_64F,0,1, ksize=3)
    I1y = cv2.Sobel(green,cv2.CV_64F,0,1, ksize=3)
    I2y = cv2.Sobel(red,cv2.CV_64F,0,1, ksize=3)

    return I0x,I1x,I2x,I0y,I1y,I2y


In [41]:
def calcOpticalFlowLK_Multi_Channel(prevImg, nextImg, winSize = 39, threshD = 1e-9):
    prevImg = prevImg/1.
    nextImg = nextImg/1.

    #PrevImgBlue,PrevImgGreen,PrevImgrRed = extractChannels(prevImg)
    #NextImgBlue,NextImgGreen,NextImgrRed = extractChannels(prevImg)

    #Calculate derivatives alond x and y

    prevI0x,prevI1x,prevI2x,prevI0y,prevI1y,prevI2y = calcDerivativesForEachChannel(prevImg)
    nextI0x,nextI1x,nextI2x,nextI0y,nextI1y,nextI2y = calcDerivativesForEachChannel(nextImg)


    #compromise solution by the least squares principle

    sigma1 = prevI0x**2 + prevI1x**2 + prevI2x**2+ nextI0x**2 +nextI1x**2 +nextI2x**2

    sigma2 = sigma4 = prevI0x* prevI0y + prevI1x* prevI1y + prevI2x* prevI2y + nextI0x* nextI0y + nextI1x* nextI1y + nextI2x* nextI2y

    sigma3 = (nextImg[:,:,0]-prevImg[:,:,0])*(prevI0x + nextI0x) + (nextImg[:,:,1]-prevImg[:,:,1])*(prevI1x + nextI1x)+ (nextImg[:,:,2]-prevImg[:,:,2])*(prevI2x + nextI2x)

    sigma5 = prevI0y**2 + prevI1y**2 + prevI2y**2+ nextI0y**2 +nextI1y**2 +nextI2y**2
    
    sigma6 = (nextImg[:,:,0]-prevImg[:,:,0])*(prevI0y + nextI0y) + (nextImg[:,:,1]-prevImg[:,:,1])*(prevI1y + nextI1y)+ (nextImg[:,:,2]-prevImg[:,:,2])*(prevI2y + nextI2y)

    sigma = [sigma1, sigma2, sigma3, sigma4, sigma5, sigma6]

    for i in range(len(sigma)):
        #apply average blurring
        #sigma[i] = cv2.blur(sigma[i], (winSize, winSize))
        #Gaussian blurring. The sigma determines the contribution of neighbours.
        sigma[i] = cv2.GaussianBlur(sigma[i],(winSize, winSize),0)

    #Use Cramer's rule to solve the equation

    #Calculate determinant of the coefficients
    D = sigma[1]**2 - sigma[0]*sigma[4]
    D[np.abs(D) < threshD] = np.Inf


    #Cramers rule
    # https://pressbooks.bccampus.ca/algebraintermediate/chapter/solve-systems-of-equations-using-determinants/
    # Replace the X coefficients with the constants and take the determinant 
    hx = (sigma[4]*sigma[2]-sigma[1]*sigma[5]) / D
    # Replace the Y coefficients with the constants and take the determinant 
    hy = (sigma[1]*sigma[2]-sigma[0]*sigma[5]) / D

    #length -> euclidean distance
    arrow = np.sqrt(hx**2+hy**2)

    return hx, -hy, arrow

In [35]:
def getCoor(hx, hy, arrow, step = 20, percent = 0.1):
    y, x= np.meshgrid(np.arange(0, hx.shape[0], step), np.arange(0, hx.shape[1], step))
    coor = np.vstack((x.flatten(), y.flatten()))

    arrow = arrow[y, x]
    index = np.argsort(arrow.flatten())
    coor = coor[:, index[int(-len(index)*percent):]]
    return coor

In [36]:
def drawArrow(img, hx, hy, coor, scale=50):
    color = (0, 255, 255)
    mask = np.zeros_like(img)
    m = np.max((np.abs(hx), np.abs(hy)))
    if m < 0.5:
        scale /=0.5
    for i in range(coor.shape[1]):
        x1 = int(coor[0, i])
        y1 = int(coor[1, i])
        x2 = int(coor[0, i]+hx[y1, x1]*scale)
        y2 = int(coor[1, i]+hy[y1, x1]*scale)
        mask = cv2.line(mask, (x1, y1), (x2, y2), color, thickness=1)
        mask = cv2.circle(mask,(x1,y1), 3, color, -1)
    out = cv2.add(img, mask)
    return out

In [37]:
def run_color(prevImg, nextImg):
    prevRGB = cv2.cvtColor(prevImg, cv2.COLOR_BGR2RGB)
    nextRGB = cv2.cvtColor(nextImg, cv2.COLOR_BGR2RGB)
    hx, hy, arrow= calcOpticalFlowLK_Multi_Channel(prevRGB, nextRGB)
    coor = getCoor(hx, hy, arrow)
    out = drawArrow(prevImg, hx, hy, coor)
    return out

In [38]:
def main_color(folder_name):
    folder_in = 'dataset/other-data-color/'+folder_name+'/'
    folder_out = 'results/result-other-color-LK/'+folder_name+'/'
    if not os.path.exists(folder_out):
        os.mkdir(folder_out)
    files = os.listdir(folder_in)
    files.sort()
    for i in range(len(files)-1):
        prevImg = cv2.imread(folder_in+files[i])
        nextImg = cv2.imread(folder_in+files[i+1])
        out = run_color(prevImg, nextImg)
        cv2.imwrite(folder_out+str(i)+'.png', out)

In [9]:
def calc_all():
    folders = ["Army","Backyard","Basketball","Dumptruck","Evergreen","Grove","Mequon","Schefflera","Teddy","Urban","Wooden","Yosemite"]
    for folder in folders:
        main_color(folder)
        print(folder +": Succesfully calcualted optical flow")

In [50]:
#calc_all()

Army: Succesfully calcualted optical flow
Backyard: Succesfully calcualted optical flow
Basketball: Succesfully calcualted optical flow
Dumptruck: Succesfully calcualted optical flow
Evergreen: Succesfully calcualted optical flow
Grove: Succesfully calcualted optical flow
Mequon: Succesfully calcualted optical flow
Schefflera: Succesfully calcualted optical flow
Teddy: Succesfully calcualted optical flow
Urban: Succesfully calcualted optical flow
Wooden: Succesfully calcualted optical flow
Yosemite: Succesfully calcualted optical flow


In [39]:
def calc_all_other():
    folders = ["Beanbags","Dimetrodon","DogDance","Grove2","Grove3","Hydrangea","MiniCooper","RubberWhale","Urban2","Urban3","Venus","Walking"]
    for folder in folders:
        main_color(folder)
        print(folder +": Succesfully calcualted optical flow")

In [40]:
#calc_all_other()

Beanbags: Succesfully calcualted optical flow
Dimetrodon: Succesfully calcualted optical flow
DogDance: Succesfully calcualted optical flow
Grove2: Succesfully calcualted optical flow
Grove3: Succesfully calcualted optical flow
Hydrangea: Succesfully calcualted optical flow
MiniCooper: Succesfully calcualted optical flow
RubberWhale: Succesfully calcualted optical flow
Urban2: Succesfully calcualted optical flow
Urban3: Succesfully calcualted optical flow
Venus: Succesfully calcualted optical flow
Walking: Succesfully calcualted optical flow


In [ ]:
def video(infile, outfile = 'result.avi'):
    assert os.path.exists(infile), "video doesn't exist"
    cap = cv2.VideoCapture(infile)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    out_video = cv2.VideoWriter(outfile, fourcc, fps, (int(width), int(height)))
    _, old_frame = cap.read()
    while 1:
        ret, frame = cap.read()
        if ret:
            out = run(old_frame, frame)
            cv2.imshow('frame', out)
            out_video.write(out)
        else: break
    cv2.destroyAllWindows()
    out_video.release()
    cap.release()

In [ ]:
#video("/Users/xavierverbrugge/Documents/School/Master/2e Master/1 SEM/Computer Vision/Assignment 2/Forest_15_3b_Videvo.mov")

In [ ]:
# Calculation of Average end-point error
# End-to-end point error is calculated by comparing an estimated optical flow vector (v_est ) with a groundtruth optical flow vector ( v_gt ).